In [41]:
import gmaps
import pandas as pd
import requests
import json
import time
import numpy as np

In [2]:
from config import gkey

In [3]:
#Configure gmaps
gmaps.configure(api_key=gkey)

In [4]:
#Bring in weather dataframe
weather_df = pd.read_csv('../VacationPy/weather_output_duplicate.csv')
weather_df.dropna(inplace=True)
weather_df.head()
cleaned_weather_df = weather_df[['City','Latitude','Longitude','Temp.(F)','Humidity (%)','Cloudiness(%)','Wind Speed(mph)','Max Temp']]
cleaned_weather_df

,City,Latitude,Longitude,Temp.(F),Humidity (%),Cloudiness(%),Wind Speed(mph),Max Temp
0,nikolskoye,59.70,30.79,57.31,67,38,2.24,59.00
1,bengkulu,-3.80,102.27,85.06,68,75,4.45,85.06
2,matata,-37.88,176.75,57.49,86,100,5.01,57.99
3,cayenne,4.93,-52.33,75.20,100,76,7.18,75.20
4,barrow,71.29,-156.79,48.20,70,40,3.36,48.20
...,...,...,...,...,...,...,...,...
596,jiazi,22.88,116.07,86.54,74,100,9.82,86.54
597,bukachacha,52.98,116.92,58.01,90,100,4.03,58.01
598,jalu,29.03,21.55,81.54,36,0,10.74,81.54
599,gbarnga,7.00,-9.47,62.80,98,47,2.08,62.80


In [5]:
#Store Lat and Long in locations
locations = cleaned_weather_df[['Latitude', 'Longitude']]
locations

,Latitude,Longitude
0,59.70,30.79
1,-3.80,102.27
2,-37.88,176.75
3,4.93,-52.33
4,71.29,-156.79
...,...,...
596,22.88,116.07
597,52.98,116.92
598,29.03,21.55
599,7.00,-9.47


In [6]:
humidity = cleaned_weather_df['Humidity (%)']
humidity

0       67
1       68
2       86
3      100
4       70
      ... 
596     74
597     90
598     36
599     98
600     51
Name: Humidity (%), Length: 601, dtype: int64

In [7]:
#Plot heat map of humidity
# set up figure information first
fig = gmaps.figure()

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}


In [8]:
#add heatmap layer to figure using gmaps
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False,
                                max_intensity = 10, point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
#Max temp lower than 80 degrees but higher than 70
ideal_weather_df = cleaned_weather_df.loc[(cleaned_weather_df['Max Temp'] > 70) & (cleaned_weather_df['Max Temp'] < 80)]
 
#Wind Speed less than 10 mph
ideal_weather_df = cleaned_weather_df.loc[(cleaned_weather_df['Wind Speed(mph)'] < 10)]

#Zero cloudiness
ideal_weather_df = cleaned_weather_df.loc[(cleaned_weather_df['Cloudiness(%)'] == 0)]

#Drop any rows with null values
ideal_weather_df = cleaned_weather_df.dropna()

ideal_weather_df

,City,Latitude,Longitude,Temp.(F),Humidity (%),Cloudiness(%),Wind Speed(mph),Max Temp
0,nikolskoye,59.70,30.79,57.31,67,38,2.24,59.00
1,bengkulu,-3.80,102.27,85.06,68,75,4.45,85.06
2,matata,-37.88,176.75,57.49,86,100,5.01,57.99
3,cayenne,4.93,-52.33,75.20,100,76,7.18,75.20
4,barrow,71.29,-156.79,48.20,70,40,3.36,48.20
...,...,...,...,...,...,...,...,...
596,jiazi,22.88,116.07,86.54,74,100,9.82,86.54
597,bukachacha,52.98,116.92,58.01,90,100,4.03,58.01
598,jalu,29.03,21.55,81.54,36,0,10.74,81.54
599,gbarnga,7.00,-9.47,62.80,98,47,2.08,62.80


In [21]:
hotels_df = ideal_weather_df.loc[:,["City","Latitude","Longitude"]].head(50)
hotels_df["Hotel Name"] = ""
hotels_df

,City,Latitude,Longitude,Hotel Name
0,nikolskoye,59.70,30.79,
1,bengkulu,-3.80,102.27,
2,matata,-37.88,176.75,
3,cayenne,4.93,-52.33,
4,barrow,71.29,-156.79,
5,ribeira grande,38.52,-28.70,
6,kolosovka,56.47,73.61,
7,avarua,-21.21,-159.78,
8,albany,42.60,-73.97,
9,isangel,-19.55,169.27,


In [26]:
#set up parameters dictionary
params = {
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotel",
    "key":gkey
}

for index, row in hotels_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city = row["City"]

    #Set coordinates for each iteration
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #Put all together to form URL and make API call
    print(f"Retrieving Results for Index {index}:{city}." )
    response = requests.get(base_url, params=params).json()
    
    #Retrieve Results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotels_df.loc[index, "Hotel Name"] = results[0]['name']
    
    except (KeyError, IndexError):
        print("Missing Field/result...skipping.")
    
    print("---------------------")
        
    time.sleep(1)

print("-------- Search Completed--------")
    


Retrieving Results for Index 0:nikolskoye.
Missing Field/result...skipping.
---------------------
Retrieving Results for Index 1:bengkulu.
Closest hotel is Mercure Bengkulu.
---------------------
Retrieving Results for Index 2:matata.
Closest hotel is Matata Hotel.
---------------------
Retrieving Results for Index 3:cayenne.
Closest hotel is Royal Amazonia.
---------------------
Retrieving Results for Index 4:barrow.
Closest hotel is Top of the World Hotel.
---------------------
Retrieving Results for Index 5:ribeira grande.
Closest hotel is Quinta Das Buganvílias.
---------------------
Retrieving Results for Index 6:kolosovka.
Closest hotel is Kolosovskaya Gostinitsa.
---------------------
Retrieving Results for Index 7:avarua.
Closest hotel is Manuia Beach Resort.
---------------------
Retrieving Results for Index 8:albany.
Missing Field/result...skipping.
---------------------
Retrieving Results for Index 9:isangel.
Closest hotel is Quarumanu area.
---------------------
Retrieving 

In [43]:
hotels_df['Hotel Name'].replace('', np.nan, inplace=True)
hotels_df.dropna(subset=['Hotel Name'], inplace = True)
hotels_df

,City,Latitude,Longitude,Hotel Name
1,bengkulu,-3.80,102.27,Mercure Bengkulu
2,matata,-37.88,176.75,Matata Hotel
3,cayenne,4.93,-52.33,Royal Amazonia
4,barrow,71.29,-156.79,Top of the World Hotel
5,ribeira grande,38.52,-28.70,Quinta Das Buganvílias
6,kolosovka,56.47,73.61,Kolosovskaya Gostinitsa
7,avarua,-21.21,-159.78,Manuia Beach Resort
9,isangel,-19.55,169.27,Quarumanu area
10,qaanaaq,77.48,-69.36,Qaanaaq Hotel
11,pevek,69.70,170.31,Munitsipal'naya Gostinitsa


In [44]:
#Assign Marker Layer
locations_2 = hotels_df[['Latitude','Longitude']]
markers = gmaps.marker_layer(locations_2)

#Template to add hotel marks
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
#Store Dataframe Row
hotels_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]

#Add Marker layer and info box
markers = gmaps.marker_layer(locations_2, info_box_content = hotels_info)

#Add layer to map
fig.add_layer(markers)

#display
fig


Figure(layout=FigureLayout(height='420px'))